In [1]:
from datetime import datetime

import polars as pl
from lumibot.backtesting import YahooDataBacktesting
from lumibot.brokers import Alpaca
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader

from humbldata.core.utils.env import Env
from humbldata.toolbox.toolbox_controller import Toolbox


2024-12-10 15:52:23 | lumibot.credentials | WARNING | No .env file found. This is ok if you are using environment variables or secrets (like on Replit, AWS, etc), but if you are not, please create a .env file in the root directory of the project.
2024-12-10 15:52:23 | asyncio | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
2024-12-10 15:52:23 | alpaca.trading.stream | INFO | started trading stream
2024-12-10 15:52:23 | alpaca.trading.stream | INFO | starting trading websocket connection
INFO: alpaca.trading.stream || connected to: BaseURL.TRADING_STREAM_PAPER


In [2]:
class HumblChannelSingleLong(Strategy):
    """Strategy that trades based on HumblChannel indicators."""

    parameters: ClassVar[dict[str, Any]] = {
        "symbol": "SPY",
        "quantity": 1,
        "buy_threshold_pct": 0.02,  # 2% threshold
        "sell_threshold_pct": 0.02,  # 2% threshold
    }

    def initialize(self):
        self.sleeptime = "1D"

    def before_market_opens(self):
        current_date = self.get_round_day()
        # current_date = self.get_datetime().strftime("%Y-%m-%d")
        # self.log_message(f"Current Trading Day: {current_date}", color="blue")

        # Calculates the humblCHANNEL from 2000-01-01 to current date, returns only one observation
        self.vars.daily_humbl_channel = (
            Toolbox(
                symbols=self.parameters["symbol"],
                interval="1d",
                start_date="2000-01-01",
                end_date=current_date,
                membership="admin",
            )
            .technical.mandelbrot_channel(
                historical=False,
                window="1mo",
                _boundary_group_down=False,
                chart=False,
                live_price=False,
            )
            .to_polars()
        )

        # self.log_message(
        #     f"HumblChannel Bottom Price: {self.vars.daily_humbl_channel.select('bottom_price')}",
        #     color="blue",
        # )
        # self.log_message(
        #     f"HumblChannel Top Price: {self.vars.daily_humbl_channel.select('top_price')}",
        #     color="blue",
        # )

    def on_trading_iteration(self):
        symbol = self.parameters["symbol"]
        cash = self.get_cash()

        # Get current price and position
        current_price = self.get_last_price(symbol)
        position = self.get_position(symbol)

        # Get bottom and top prices from HumblChannel
        bottom_price = self.vars.daily_humbl_channel.select("bottom_price").row(
            0
        )[0]
        top_price = self.vars.daily_humbl_channel.select("top_price").row(0)[0]

        # Calculate price differences as percentages
        bottom_diff_pct = abs(current_price - bottom_price) / bottom_price
        top_diff_pct = abs(current_price - top_price) / top_price

        # First check sell condition - if we have a position and price is near top
        if (
            position is not None
            and abs(top_diff_pct) <= self.parameters["sell_threshold_pct"]
        ):
            # Sell all shares
            order = self.create_order(symbol, position.quantity, "sell")
            self.submit_order(order)
            return  # Exit to prevent buy on same day

        # Then check buy condition - if we have no position and price is near bottom
        if (
            position is None
            and abs(bottom_diff_pct) <= self.parameters["buy_threshold_pct"]
        ):
            # Calculate maximum shares we can buy with available cash
            max_shares = int(cash / current_price)
            if max_shares > 0:
                order = self.create_order(symbol, max_shares, "buy")
                self.submit_order(order)


In [3]:
# Set up backtesting
env = Env()
ALPACA_CONFIG = {
    # Put your own Alpaca key here:
    "API_KEY": env.ALPACA_API_KEY,
    # Put your own Alpaca secret here:
    "API_SECRET": env.ALPACA_API_SECRET,
    # Set this to False to use a live account
    "PAPER": True
}

trader = Trader()
broker = Alpaca(ALPACA_CONFIG)

strategy = HumblChannelSingleLong(
    name="HumblChannelSingleLong",
    broker=broker,
    parameters={
        "symbol": "SPY",
        "quantity": 1,
    }
)

# Run backtest
backtesting_start = datetime(2010, 1, 1)
backtesting_end = datetime(2024, 1, 1)

results = strategy.run_backtest(
    YahooDataBacktesting,
    backtesting_start,
    backtesting_end,
)

INFO: asyncio || [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
INFO: alpaca.trading.stream || started trading stream
INFO: alpaca.trading.stream || starting trading websocket connection


INFO: asyncio || Starting backtest...
INFO: lumibot.traders.trader || Backtesting starting...
2024-12-10 15:52:41,467: INFO: connected to: BaseURL.TRADING_STREAM_PAPER


Progress ||   0.07%  [Elapsed: 0:00:00 ETA: 0:13:56] 

2024-12-10 15:52:41,983: INFO: Current backtesting datetime 2010-01-04 08:30:00-05:00


Progress ||   0.07%  [Elapsed: 0:00:02 ETA: 0:53:25] 

...

2024-12-10 16:05:01,430: INFO: Current backtesting datetime 2010-03-16 15:25:00-05:00


Progress ||   1.46%  [Elapsed: 0:12:19 ETA: 13:52:29] 

2024-12-10 16:05:01,431: INFO: Current backtesting datetime 2010-03-16 15:25:00-05:00


Progress ||   1.47%  [Elapsed: 0:12:19 ETA: 13:44:58] 

2024-12-10 16:05:01,433: INFO: Current backtesting datetime 2010-03-17 07:30:00-05:00
